# Import necessary libraries for data handling

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.io import savemat
from tqdm import tqdm, trange  # For displaying progress bars
from copy import deepcopy
import syncopy as spy

# Import Allen SDK for accessing the EcephysProjectCache (electrophysiology data)

In [3]:
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

# Specify the path for the JSON manifest file (required by Allen SDK for file management)
# The data in the JSON file determines where downloaded data will be stored. See Allen SDK

In [4]:
manifest_path = os.path.join(os.path.curdir, 'manifest.json')
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)
sessions = cache.get_session_table()

# Define data parameters for the analysis

In [5]:
stimType = "flashes"  # 'flashes' 'drifting_gratings'  'gabors'
bipolar = 1  # 1:bipolar derivation   0:unipolar
sample_freq = 1250
sample_freq_rawdata = 30_000

# Select a session to analyze based on its index in the session table

In [6]:
sesnum = 7
ses_id = sessions.index.values[sesnum]
session = cache.get_session_data(ses_id)
probes = cache.get_probes()

# Retrieve presentation times for the specified stimulus type (Cut trials)

In [ ]:
print(pd.unique(session.stimulus_presentations.stimulus_name))
presentation_table = session.stimulus_presentations[
    session.stimulus_presentations.stimulus_name == stimType]
presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
win = np.arange(-.25, .25, 1 / sample_freq) # Define time window around each stimulus presentation
stim_lbl = session.get_stimulus_table([stimType])['color'] # Get stimulus labels
prob_range = session.probes.index.values # Define range of probes in the session

# Read Allen data and save the spikes in Syncopy file format

In [9]:
def read_spy_spikes():
    """
    Function to extract spike data from Allen SDK, process it, and save it in Syncopy file format.
    """
    print("Creating Syncopy spike data file...")

    offset = -0.25 * sample_freq  # Offset in samples for each trial

    # first assemble units of interest based on firing rate 
    units_oi = []
    for prob_ind in tqdm(prob_range):
        units_oi += list(session.units[(session.units.probe_id == prob_ind) &
                                       (session.units.firing_rate > 10) &
                                       (session.units.nn_hit_rate > 0.95)].index)

    # Loop over trials for each stimulus presentation
    spy_spikes = []
    trldef_rows = []
    active_units = []
    for stim_time in tqdm(presentation_times):

        start = stim_time - 0.25
        stop = stim_time + 0.25

        # 2d array with columns: samples, channel, unit.
        # Channel is not given/interesting for spike data and has to be 0.
        channel_id = 0

        spy_trl = []

        for unit_idx, unit_id in enumerate(units_oi):

            unit_spike_times = session.spike_times[unit_id]
            # index time window
            bool_index = (unit_spike_times > start) & (unit_spike_times < stop)
            # relevant spikes in samples
            unit_spike_samples = (
                unit_spike_times[bool_index] + start) * sample_freq

            # Skip units without spikes in this window
            if unit_spike_samples.size == 0:
                continue

            active_units.append(unit_id)

            spy_trl.append(np.array([(sp_sample, channel_id, unit_idx) for
                                     sp_sample in unit_spike_samples], dtype=np.int64))

        spy_trl = np.vstack(spy_trl)
        spy_trl = spy_trl[spy_trl[:, 0].argsort()] # Sort spike samples by time
        spy_spikes.append(spy_trl)

        trldef_rows.append([spy_trl[0, 0], spy_trl[-1, 0], offset])

    trldef = np.vstack(trldef_rows)
    spy_spikes = spy.SpikeData(data=spy_spikes, samplerate=sample_freq)

    spy_spikes.trialdefinition = trldef
    # unit labels have to be strings
    spy_spikes.unit = list(map(str, set(active_units)))

    # Add custom metadata: raw sampling rate.
    spy_spikes.info['fsample_raw'] = sample_freq_rawdata

    # Finally save to disc.
    spy_spikes.save(os.path.join(
        os.path.curdir, 'allen_spike'), overwrite=True)
    # return Spikes

# Read Allen data and save the LFP data in Syncopy file format

In [10]:
def read_spy_lfp():
    """
    Function to extract LFP data, process it for each stimulus presentation, and save it in Syncopy file format.
    """
    print("Creating Syncopy LFP data file...")

    loc = []            # Channel locations
    ds = []             # LFP data for each trial
    twin = []           # Time windows for each trial
    lfps = []
    lfp_channel_id = [] # Channel IDs for bipolar or unipolar data

    # Attempt to retrieve LFP data for each probe
    for prob_ind in tqdm(prob_range):
        try:
            lfps.append(session.get_lfp(prob_ind))
        except:
            print(f"No LFP in this electrode:{prob_range}")
            continue

    for tr in trange(len(presentation_times)):
        lfp_buf = [] # Buffer to store LFP data for the current trial

        # Append LFP of all channels
        for _, lfp in enumerate(lfps):
            ch_range = lfp.channel.values
            for z in range(len(ch_range)):
                channel_ids = ch_range[z]
                try:
                    # Retrieve bipolar or unipolar LFP data depending on the parameter set
                    if bipolar == 1:
                        lfp_tmp = (lfp.sel(time=(presentation_times[tr]+win), method='nearest', channel=ch_range[z]))-(
                            lfp.sel(time=(presentation_times[tr]+win), method='nearest', channel=ch_range[z+1]))
                        loc_tmp = (
                            session.channels.ecephys_structure_acronym[channel_ids])
                    elif bipolar == 0:
                        lfp_tmp = (lfp.sel(
                            time=(presentation_times[tr]+win), method='nearest', channel=channel_ids))
                        loc_tmp = (
                            session.channels.ecephys_structure_acronym[channel_ids])

                    if tr == 0:
                        loc.append(loc_tmp)
                        if bipolar == 1:
                            lfp_channel_id.append([ch_range[z], ch_range[z+1]])
                        elif bipolar == 0:
                            lfp_channel_id.append(channel_ids)
                    lfp_buf.append(lfp_tmp)
                    lfp_buf[-1] = np.nan_to_num(
                        lfp_buf[-1].values.reshape(1, -1))
                except:
                    pass

        ds.append(np.concatenate(lfp_buf, axis=0).T)
        twin.append(deepcopy(win))

    # Put data into Syncopy objects.
    lfp = spy.AnalogData(data=ds, samplerate=sample_freq)
    lfp.info['Label'] = loc
    lfp.info['stim_lbl'] = list(stim_lbl)
    lfp.info['Twin'] = [arr.tolist() for arr in twin]

    # Finally save to disc.
    lfp.save(os.path.join(os.path.curdir, 'allen_lfp'), overwrite=True)
    # return LFP

# Read Allen data and save data in Fieldtrip-compatible (MATLAB) format

In [ ]:
def read_ft():
    """
    Function to extract both spike and LFP data, process it, and save it in Fieldtrip-compatible format.
    """
    print("Creating Fieldtrip data file...")

    spikes = []
    loc = []
    locspk = []
    ds = []
    ds_spk = []
    twin = []
    lfps = []
    spk_channel_id = []
    lfp_channel_id = []

    fieldtrip = {"Label": [],  # cell(1*chNum)
                 "Labelspk": [],
                 "chIDlfp": [],
                 "chIDspk": [],
                 "trialinfo": {},  # double(trialNum*info)
                 "trial": [],          # cell(1*trialNum) >> (chNum*sample)
                 "trial_spike": [],
                 "time": [],           # cell(1*trialNum) >> (1*sample)
                 "fsample": sample_freq,        # sampling rate  1250
                 "fsample_raw": sample_freq_rawdata,
                 # 1*1 structure (other infor) >> not necessary
                 "cfg": []
                 }

    # Append spikes of all channels
    for prob_ind in tqdm(prob_range):
        try:
            lfps.append(session.get_lfp(prob_ind))
        except:
            continue
        units_of_interest = (session.units[(session.units.probe_id == prob_ind) &
                                           (session.units.firing_rate > 10) &
                                           (session.units.nn_hit_rate > 0.95)])
        for i in range(len(units_of_interest)):
            unit_id = units_of_interest.index.values[i]
            channel_index = units_of_interest .loc[unit_id].probe_channel_number
            spk_channel_id.append(session.channels[(session.channels.probe_channel_number == channel_index) &
                                                   (session.channels.probe_id == prob_ind)].index.values[0])
            spikes.append(session.spike_times[unit_id])

    for tr in trange(len(presentation_times)):
        lfp_buf = []
        spk_buf = []
        for unitNum in range(len(spikes)):
            tmp = np.zeros(len(win))
            sp = (spikes[unitNum][(spikes[unitNum] > presentation_times[tr]+win[0]) &
                                  (spikes[unitNum] < presentation_times[tr]+win[-1])])-(presentation_times[tr]+win[0])
            for i in sp:
                tmp[int(i*fieldtrip['fsample'])] = 1
            spk_buf.append(tmp)
            if tr == 0:
                locspk.append(
                    session.channels.ecephys_structure_acronym[spk_channel_id[unitNum]])

        # Append LFP of all channels
        for _, lfp in enumerate(lfps):
            ch_range = lfp.channel.values
            for z in range(len(ch_range)):
                channel_ids = ch_range[z]
                try:
                    if bipolar == 1:
                        lfp_tmp = (lfp.sel(time=(presentation_times[tr]+win), method='nearest', channel=ch_range[z]))-(
                            lfp.sel(time=(presentation_times[tr]+win), method='nearest', channel=ch_range[z+1]))
                        loc_tmp = (
                            session.channels.ecephys_structure_acronym[channel_ids])
                    elif bipolar == 0:
                        lfp_tmp = (lfp.sel(
                            time=(presentation_times[tr]+win), method='nearest', channel=channel_ids))
                        loc_tmp = (
                            session.channels.ecephys_structure_acronym[channel_ids])
                    if tr == 0:
                        loc.append(loc_tmp)
                        if bipolar == 1:
                            lfp_channel_id.append([ch_range[z], ch_range[z+1]])
                        elif bipolar == 0:
                            lfp_channel_id.append(channel_ids)
                    lfp_buf.append(lfp_tmp)
                    lfp_buf[-1] = np.nan_to_num(
                        lfp_buf[-1].values.reshape(1, -1))
                except:
                    pass

        ds.append(np.concatenate(lfp_buf, axis=0).T)
        print(ds[-1].shape)
        twin.append(deepcopy(win))
        ds_spk.append(list(map(list, zip(*spk_buf))))
    
    # Save the data in Fieldtrip-compatible format
    fieldtrip["trial"] = ds
    fieldtrip["spike"] = ds_spk
    fieldtrip["Label"] = loc
    fieldtrip["Labelspk"] = locspk
    fieldtrip["time"] = twin
    fieldtrip["trialinfo"] = {'stim_lbl': stim_lbl,
                              'chIDlfp': lfp_channel_id,
                              'chIDspk': spk_channel_id
                              }
    fieldtrip2 = deepcopy(fieldtrip)
    fieldtrip2["trialinfo"] = []
    fieldtrip2["stim_lbl"] = list(stim_lbl)

    savemat(os.path.join(os.path.curdir, 'allen_FT.mat'), fieldtrip2)

# Execute the functions if running as a script
if __name__ == "__main__":
    read_spy_spikes()
    read_spy_lfp()
    read_ft()

# Time-lock raster plot and peri-stimulus time histogram of spiking activity

# Import necessary libraries for data handling

In [13]:
import syncopy as spy
import matplotlib.pyplot as plt

# set parameters and load spike data

In [14]:
plt.rcParams['pdf.fonttype'] = 42
saveFig = 1             # 0:not savefigs  1: save figs
spikeData = spy.load('allen_spike.spy')
data_spk = spikeData.trials
samplerate = 1250
offset = -0.25 * samplerate  # Offset in samples for aligning samples to stimulus onset.

# Raster plot

In [ ]:
unitID = '951009344'
unitname = 'unit30'
plt.close('all')
plt.figure()
spikeData.singlepanelplot(unit=unitID)

if saveFig == 1:
    plt.savefig("Raster.png")
    plt.savefig("Raster.pdf")

# PSTH

In [ ]:
cfg = spy.StructDict()
cfg.latency = "minperiod"  # to avoid NaNs
cfg.output = "rate"
cfg.binsize = 0.01  # in seconds
spike_rate = spy.spike_psth(spikeData, cfg)

# Remove channel0 from channel names.
chan_names = [old_name.split('_')[1] for old_name in spike_rate.channel]
spike_rate.channel = chan_names

# Compute the mean.
rate_trl_mean = spy.mean(spike_rate, dim='trials')

# Plot.
fig, ax = rate_trl_mean.singlepanelplot(channel=unitname)
ax.set_ylabel('spike rate (1/s)')
fig.tight_layout()

if saveFig == 1:
    plt.savefig("PSTH.png")
    plt.savefig("PSTH.pdf")

# Power spectrum of LFP

# Import necessary libraries for data handling

In [17]:
import pickle
import numpy as np
import syncopy as spy
import matplotlib.pyplot as plt

# set parameters and load LFP data

In [18]:
# Parameters
plt.rcParams['pdf.fonttype'] = 42
saveFig = 1             # 0:not savefigs  1: save figs
freq_range = [15, 90]   # frequency range
fs = 1250               # sampling rate
areas = {
    "V1L": ['VISl'],
    "V1RL": ['VISrl'],
    "Thal-MG": ['MGv'],
    "Thal-LP": ['LP']
}

# Load data
lfpdata = spy.load('allen_lfp.spy')

# Power estimation

In [ ]:
# select stimulus and baseline epochs for analysis
L = len(lfpdata.time[0])
base_time = np.arange(0, np.ceil(L/2)-1, 1)
stim_time = np.arange(np.floor(L/2), L-1, 1)
TW_base = [lfpdata.time[0][int(base_time[0])],
           lfpdata.time[0][int(base_time[-1])]]
TW_stim = [lfpdata.time[0][int(stim_time[0])],
           lfpdata.time[0][int(stim_time[-1])]]
data_base = lfpdata.selectdata(latency=TW_base)
data_stim = lfpdata.selectdata(latency=TW_stim)

# Initialize configuration settings for Syncopy power analysis
cfg = []
cfg = spy.get_defaults(spy.freqanalysis)
cfg.method = 'mtmfft'
cfg.output = 'pow'
cfg.pad = 1
cfg.taper = 'hann'
cfg.nTaper = 1
cfg.keeptrials = True
cfg.foilim = freq_range

# Baseline correction
psd_stim = spy.freqanalysis(cfg, data_stim)
psd_base = spy.freqanalysis(cfg, data_base)
psd = []
psd2 = []
psd = psd_stim
powbs = (np.mean(psd_base.data, axis=0))
psd.data = psd_stim.data/powbs
selectch = np.arange(0, len(psd.channel), 1)
pow = psd.data

# Outlier detection
outl = np.nanmean(np.nanmean(pow, axis=2), axis=0) > (np.nanmean(
    np.reshape(pow, (1, -1)), axis=1)+3*np.std(np.reshape(pow, (1, -1)), axis=1))
indices = np.where(outl)[0]
selch = np.delete(selectch, indices)
spy.selectdata(psd, channel=list(lfpdata.channel[selch]))

cfg = spy.StructDict()
psd2 = spy.selectdata(cfg, psd)

# Grand power
channelLBL = lfpdata.info['Label']
GrandPSD = spy.mean(psd2, dim='trials')
iter = 1
Pow = []

# for area_iter in areas.keys():
area_iter = list(areas.keys())[0]
area = []
y = []
y_tmp = []
indexes = []
for i in areas[f"{area_iter}"]:
    indexes.append([index for index in range(
        len(channelLBL)) if channelLBL[index] == i])
chindx = list(map(int, np.concatenate(indexes)))

y_tmp = spy.selectdata(GrandPSD, channel=chindx)
y = spy.mean(y_tmp, dim='channel')
fig, ax = y.singlepanelplot(logscale=False)
ax.set_title(f"ch= {area_iter}")
ax.set_xlabel('Frequency(Hz)')
ax.set_ylabel('Syncopy power')
ax.set_xlim([1, 90])
Pow.append(np.squeeze(y.data))
iter = iter+1

if saveFig == 1:
    plt.savefig(f'power{area_iter}.png')
    plt.savefig(f'power{area_iter}.pdf')

pickle.dump(Pow, open(f"SavedData/PyPow.plk", "wb"))

# Connectivity analysis

# Import necessary libraries for data handling

In [20]:
import pickle
import numpy as np
import syncopy as spy
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

# set parameters and load LFP data

In [21]:
plt.rcParams['pdf.fonttype'] = 42
cohppcGC = 1  # 1: coh  2:ppc 3:GC
saveFig = 1             # 0:not savefigs  1: save figs
freq_range = [1, 100]   # frequency range
fs = 1250               # sampling rate
areas = {
    "V1L": ['VISl'],
    "V1RL": ['VISrl'],
    "Thal-MG": ['MGv'],
    "Thal-LP": ['LP']
}

# Load data
lfpdata = spy.load('allen_lfp.spy')
channelLBL = lfpdata.info['Label']

#  connectivity estimation

In [ ]:
L = len(lfpdata.time[0]) # Length of the time series for LFP data
stim_time = np.arange(np.floor(L/2), L-1, 1) # Define stimulus time range
TW_stim = [lfpdata.time[0][int(stim_time[0])],
           lfpdata.time[0][int(stim_time[-1])]] # Define time window for analysis
data_stim_all = lfpdata.selectdata(latency=TW_stim)  # Select data within the specified time window for analysis


# Initialize configuration settings for Syncopy connectivity analysis
cfg = []
cfg = spy.StructDict()
if cohppcGC == 1:
    cfg.method = 'coh' # Coherence method
elif cohppcGC == 2:
    cfg.method = 'ppc' # Pairwise phase consistency
elif cohppcGC == 3:
    cfg.method = 'granger'  # Granger causality
    np.seterr(divide='ignore', invalid='ignore')

cfg.pad = 1  # 'maxperlen'     # Padding applied to time series
# Optional configuration settings for spectral analysis (tapering and smoothing)
# cfg.taper = 'None' #'hann'  # 'dpss' 'hann'
# cfg.nTaper = 1
# cfg.tapsmofrq = 3

# Loop over defined brain areas and their channels to compute connectivity
area_num = list(areas.keys())
plt.close('all')
plt.figure()
iter = 0

# Iterate over pairs of areas (sender and receiver) to calculate connectivity metrics
for sender in range(0, 1):  # range(len(area_num)):  # -1    # Iterate over each sender area
    for receiver in range(1, 2):  # range(len(area_num)):  # sender+1,    # Iterate over each receiver area
        iter = iter+1
        if receiver > sender:
            # For each sender and receiver pair, find and analyze relevant channels
            for sender_ch in areas[area_num[sender]]:
                for receiver_ch in areas[area_num[receiver]]:
                    Con12 = []  # Storage for sender-to-receiver connectivity
                    Con21 = []  # Storage for receiver-to-sender connectivity
                    ch1 = []  # Lists to hold the channel indices
                    ch2 = []
                    ppc_val = []  # PPC and feedback PPC values
                    ppc_val_FB = []
                    
                    # Find indices for the channels based on the labels
                    for index, item in enumerate(channelLBL):
                        if item == sender_ch:
                            ch1.append(index)
                        if item == receiver_ch:
                            ch2.append(index)
                    # ch1 = np.nonzero(channelLBL == sender_ch)[0]
                    # ch2 = np.nonzero(channelLBL == receiver_ch)[0]

                    # Iterate over channel combinations for sender and receiver
                    for c1 in ch1:
                        for c2 in ch2:
                            print(c1)
                            chsV1V2 = np.concatenate((c1, c2), axis=None)
                            data_stim_subGroup = spy.selectdata(
                                data_stim_all, channel=chsV1V2)
                            
                            # Perform connectivity analysis based on method specified in cfg
                            ppc = spy.connectivityanalysis(
                                data_stim_subGroup, cfg)
                            fr = np.where((ppc.freq >= 15) &
                                          (ppc.freq <= 90))[0]
                            fr = fr+1

                            # Identify sender and receiver indices within channel pair
                            c11 = []
                            c22 = []
                            for index, item in enumerate(chsV1V2):
                                if channelLBL[item] == sender_ch:
                                    c11.append(index)
                                if channelLBL[item] == receiver_ch:
                                    c22.append(index)
                            # ppc_val = spy.selectdata(ppc, frequency=[min(fr), max(fr)], channel_i=c11, channel_j=c22)
                            tmp1 = ppc.data[0]
                            tmp2 = tmp1[fr, c11, c22]
                            Con12.append(tmp2)
                            # # Con12.append(np.squeeze(ppc_val.data))

                            # If method is Granger causality, also calculate feedback direction
                            if cohppcGC == 3:
                                ppc_val_FB = spy.selectdata(
                                    ppc, frequency=[min(fr), max(fr)], channel_i=c22, channel_j=c11)
                                # # Con21.append(np.squeeze(ppc_val_FB.data))
                                tmp1 = []
                                tmp2 = []
                                tmp1 = ppc.data[0]
                                tmp2 = tmp1[fr, c22, c11]
                                Con21.append(tmp2)
                    
                    # Plotting connectivity values
                    print(sender, receiver)
                    plt.subplot(4, 4, iter)
                    plt.plot(np.squeeze(np.nanmean(Con12, axis=0)), '-',
                             lw=2, c='red', alpha=0.5)
                    if cohppcGC == 3:
                        plt.plot(np.squeeze(np.nanmean(Con21, axis=0)),
                                 '--', lw=2, c='red', alpha=0.5)
                        plt.title(
                            f"__{area_num[sender]}->{area_num[receiver]}")
                    else:
                        plt.title(f"{area_num[sender]},-,{area_num[receiver]}")

                    plt.xticks([], [])
                    if iter == 1:
                        plt.xlabel('Frequency(Hz)')
                        if cohppcGC == 1:
                            plt.ylabel('Coherence')
                        elif cohppcGC == 2:
                            plt.ylabel('PPC')
                        elif cohppcGC == 3:
                            plt.ylabel('GC')
                    if saveFig == 1:
                        plt.savefig(f'coh_ppc_gc_{cohppcGC}.png')
                        plt.savefig(f'coh_ppc_gc_{cohppcGC}.pdf')

# Compile connectivity results into a dictionary and save to a file
Con = {'fr': fr,
       'Con12': Con12,
       'Con21': Con21,
       }

pickle.dump(Con, open(f"SavedData/PyCohPpcGC{cohppcGC}.plk", "wb"))